# Scrape Billboard 100 for songs and Create Spotify Playlist

In [1]:
import requests as re
from bs4 import BeautifulSoup
import os

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Setting a new environment variable
os.environ['SPOTIPY_CLIENT_ID'] = '861d60b3123f4e46bb6b389cf20f8e20'
os.environ['SPOTIPY_CLIENT_SECRET'] = '852eaf36d485420aa910e44ebde7d556'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:3000'


## Billboard Site Scrape

In [2]:
#get user determined date for scrape/playlist
#date has to bee from last week or earlier
date = input("Enter date for Billboard Top 100 songs(YYYY-MM-DD):")

url = f"https://www.billboard.com/charts/hot-100/{date}"

response = re.get(url)
soup = BeautifulSoup(response.text,"html.parser")


Enter date for Billboard Top 100 songs(YYYY-MM-DD):2006-08-14


In [3]:
song_cards = soup.select('.o-chart-results-list-row-container')
song_list = []
for song in song_cards:
    track_name = song.select_one('#title-of-a-story')
    track = track_name.text.strip()
    artist_name = track_name.find_next('span').text.strip()
    song_list.append((track,artist_name))

print(f'Song List Length: {len(song_list)}')

Song List Length: 100


## Spotipy: Spotify API Library

In [4]:
#get spotify authentication and create spotify api class
#scopes limit the access an api has to user account based on desired function
scope = "playlist-modify-private"
#create class instance
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [5]:
#create playlist
play_resp = sp.user_playlist_create('jtazioli',f'Billboard Top 100, {date}',public=False,collaborative=False,description='Top 100 songs of the week according to Billboard top 100.')
PLAYLIST_ID = play_resp['id']

In [6]:
#find song URIs needed to add them to playlist
#search q: q=track%3AsongName%20artist%3AartistName
id_list = []
for entry in song_list:
    q=f'track%3{entry[0].replace(" ", "")}%20artist%3{entry[1].replace(" ", "")}'
    track_response = sp.search(q,limit=1,offset=0,type='track',market='US')
    id_list.append(track_response['tracks']['items'][0]['id'])
print(f'ID List Length: {len(id_list)}')

ID List Length: 100


In [7]:
#add songs to playlist
song_list
results = sp.user_playlist_add_tracks('jtazioli',PLAYLIST_ID,id_list)

In [8]:
play_resp

{'collaborative': False,
 'description': 'Top 100 songs of the week according to Billboard top 100.',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1PY2YLSLjJuF1G8NIzjsYy'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/1PY2YLSLjJuF1G8NIzjsYy',
 'id': '1PY2YLSLjJuF1G8NIzjsYy',
 'images': [],
 'primary_color': None,
 'name': 'Billboard Top 100, 2006-08-14',
 'type': 'playlist',
 'uri': 'spotify:playlist:1PY2YLSLjJuF1G8NIzjsYy',
 'owner': {'href': 'https://api.spotify.com/v1/users/jtazioli',
  'id': 'jtazioli',
  'type': 'user',
  'uri': 'spotify:user:jtazioli',
  'display_name': None,
  'external_urls': {'spotify': 'https://open.spotify.com/user/jtazioli'}},
 'public': False,
 'snapshot_id': 'AAABLisUTp7A/PF14Fl3TA1ciZEmH3RJ',
 'tracks': {'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'href': 'https://api.spotify.com/v1/playlists/1PY2YLSLjJuF1G8NIzjsYy/tracks',
  'total': 0,
  'items': []}}